## Create temporaray csv file for the entire year of coverage

This notebook should be the first one running. It takes all the files in `input_data` and combines them in a csv that spans through 1 year of data. The data used as source should have the same structure in order to be combined.

In [1]:
import os
import pandas as pd

## NYT Data

In [2]:
# Define the path to the input-data folder
input_data_folder = '../input-data/nyt'

# List all files in the input-data folder
files = os.listdir(input_data_folder)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(input_data_folder, file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
full_year = pd.concat(dataframes, ignore_index=True)

# Display the unique dataframe
full_year

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,subsection_name,byline,type_of_material,_id,word_count,uri
0,President-elect Donald J. Trump turned to a fi...,https://www.nytimes.com/2024/11/30/us/politics...,President-elect Donald J. Trump turned to a fi...,President-elect Donald J. Trump said on Saturd...,A,1.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Trump Says He Will Nominate Kash Pat...,"[{'name': 'subject', 'value': 'United States P...",2024-12-01T00:00:48+0000,article,Washington,U.S.,Politics,{'original': 'By Devlin Barrett and Maggie Hab...,News,nyt://article/e623b7bd-0983-5ba8-86c8-5e56024b...,1132,nyt://article/e623b7bd-0983-5ba8-86c8-5e56024b...
1,"The San Jose State women’s team, which has a t...",https://www.nytimes.com/2024/11/30/us/transgen...,"The San Jose State women’s team, which has a t...",After months at the center of a debate over a ...,D,6.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'A College Volleyball Team’s Season i...,"[{'name': 'subject', 'value': 'Volleyball', 'r...",2024-12-01T00:25:04+0000,article,National,U.S.,NaN,"{'original': 'By Juliet Macur', 'person': [{'f...",News,nyt://article/1740e8c4-2140-5971-94b2-a59b7c6b...,1265,nyt://article/1740e8c4-2140-5971-94b2-a59b7c6b...
2,"No corrections appeared in print on Sunday, De...",https://www.nytimes.com/2024/11/30/pageoneplus...,"No corrections appeared in print on Sunday, De...",Errors are corrected during the press run when...,NaN,NaN,The New York Times,[],"{'main': 'No Corrections: Dec. 1, 2024', 'kick...",[],2024-12-01T02:30:01+0000,article,Corrections,Corrections,NaN,"{'original': '', 'person': [], 'organization':...",News,nyt://article/2e856105-d452-5ec0-b494-9121372f...,55,nyt://article/2e856105-d452-5ec0-b494-9121372f...
3,The announcement of Sheriff Chad Chronister ca...,https://www.nytimes.com/2024/11/30/us/politics...,The announcement of Sheriff Chad Chronister ca...,President-elect Donald J. Trump on Saturday ch...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Trump Picks a Florida Sheriff as D.E...,"[{'name': 'persons', 'value': 'Chronister, Cha...",2024-12-01T02:40:51+0000,article,Washington,U.S.,Politics,"{'original': 'By Mark Walker', 'person': [{'fi...",News,nyt://article/357ba9c8-88c3-5904-b513-03e94c69...,393,nyt://article/357ba9c8-88c3-5904-b513-03e94c69...
4,President-elect Donald J. Trump has long seen ...,https://www.nytimes.com/2024/11/30/us/politics...,President-elect Donald J. Trump has long seen ...,"Kash Patel, President-elect Donald J. Trump’s ...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Five Things to Know About Kash Patel...,"[{'name': 'subject', 'value': 'United States P...",2024-12-01T03:22:38+0000,article,Washington,U.S.,Politics,"{'original': 'By Elizabeth Williamson', 'perso...",News,nyt://article/55b50967-d7c0-5f73-8236-0e535dd5...,693,nyt://article/55b50967-d7c0-5f73-8236-0e535dd5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24731,The brand has updated its 2003 collaboration w...,https://www.nytimes.com/2025/01/31/t-magazine/...,The brand has updated its 2003 collaboration w...,The 63-year-old Japanese artist Takashi Muraka...,M2,132.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Takashi Murakami’s Beloved, Trippy L...","[{'name': 'subject', 'value': 'Fashion and App...",2025-01-31T16:14:52+0000,article,TStyle,T Magazine,NaN,"{'original': 'By Lindsay Talbot', 'person': [{...",News,nyt://article/2fdf3dc5-710b-5253-ad61-3dc236c7...,364,nyt://article/2fdf3dc5-710b-5253-ad61-3dc236c7...
24732,"A loft in Long Beach, a Spanish-Style house in...",https://www.nytimes.com/2025/01/31/realestate/...,"A loft in Long Beach, a Spanish-Style house in...",To provide a resource for th

In [3]:
full_year.to_csv("../input-data/nyt-temp-data.csv")

## Zeit Data

In [ ]:
# Define the path to the input-data folder
input_data_folder = '../input-data/zeit'

# List all files in the input-data folder
files = os.listdir(input_data_folder)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(input_data_folder, file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
full_year = pd.concat(dataframes, ignore_index=True)

# Display the unique dataframe
full_year

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,...,body,source,authors,image,eventUri,sentiment,wgt,relevance,keywords,links
0,2024-12-590516688,deu,False,2024-12-31,23:48:05,2024-12-31 23:48:05+00:00,2024-12-31T23:35:24Z,news,0.000000,https://www.zeit.de/news/2025-01/01/feuerwehr-...,...,Kurz vor dem Jahreswechsel ist in Hamburg ein ...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/feuerwehr-...,None,NaN,473384885,1,"[News, Silvesternacht, Billstedt, Hamburg, Feu...","[{'anchorText': 'Feuerwehr', 'url': 'https://w..."
1,8481124955,deu,False,2024-12-31,23:26:31,2024-12-31 23:26:31+00:00,2024-12-31T23:25:31Z,news,0.968627,https://www.zeit.de/news/2025-01/01/zweite-squ...,...,"Das Drama um scheinbar harmlose, jedoch am End...","{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/zweite-squ...,eng-10211662,NaN,473383591,1,"[News, Netflix-Welthit, Netflix, Squid, Kinder...","[{'anchorText': 'Netflix', 'url': 'https://www..."
2,8481124174,deu,False,2024-12-31,23:25:05,2024-12-31 23:25:05+00:00,2024-12-31T23:24:01Z,news,0.000000,https://www.zeit.de/gesellschaft/zeitgeschehen...,...,Vor der Silvesternacht in Berlin hat die Poliz...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/gesellschaft/zeitgeschehen...,None,NaN,473383505,1,"[Gesellschaft, Jahreswechsel, Silvester, Feuer...","[{'anchorText': 'Polizei', 'url': 'https://www..."
3,8481124175,deu,False,2024-12-31,23:24:34,2024-12-31 23:24:34+00:00,2024-12-31T23:24:01Z,news,0.000000,https://www.zeit.de/news/2025-01/01/haus-in-hu...,...,Ein Einfamilienhaus ist in der Silvesternacht ...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/haus-in-hu...,None,NaN,473383474,1,"[News, Großeinsatz der Feuerwehr, Hude, Landkr...","[{'anchorText': 'Feuerwehr', 'url': 'https://w..."
4,8481116703,deu,False,2024-12-31,23:14:35,2024-12-31 23:14:35+00:00,2024-12-31T23:12:42Z,news,0.945098,https://www.zeit.de/news/2025-01/01/polen-uebe...,...,Polen hat zum Jahreswechsel den alle sechs Mon...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/polen-uebe...,eng-10209752,NaN,473382875,1,"[News, Nach sechs Monaten Ungarn, Donald Tusk,...","[{'anchorText': 'Polen', 'url': 'https://www.z..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64840,8105574979,deu,False,2024-05-01,02:23:35,2024-05-01 02:23:35+00:00,2024-05-01T02:22:46Z,news,0.800000,https://www.zeit.de/news/2024-05/01/jahrestag-...,...,Die Zusammenfassung für diesen Artikel kann le...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2024-05/01/jahrestag-...,deu-1781817,NaN,452226215,1,"[News, EU, Annalena Baerbock, Radosław Sikorsk...","[{'anchorText': 'Annalena Baerbock', 'url': 'h..."
64841,8105574980,deu,False,2024-05-01,02:23:18,2024-05-01 02:23:18+00:00,2024-05-01T02:22:46Z,news,0.588235,https://www.zeit.de/politik/ausland/2024-05/us...,...,Die Audioversion dieses Artikels wurde künstli...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/politik/ausland/2024-05/mi...,deu-1781792,NaN,452226198,1,"[Politik, US-Repräsentantenhaus, US-Repräsenta...",None
64842,8105568694,deu,False,2024-05-01,02:15:24,2024-05-01 02:15:24+00:00,2024-05-01T02:14:45Z,news,0.968627,https://www.zeit.de/news/2024-05/01/sek-einsat...,...,Die Zusammenfassung für diesen Artikel kann le...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2024-05/01/sek-einsat...,deu-1781828,NaN,452225724,1,"[News, Extremismus, Velbert, Düsseldorf, Poliz...","[{'anchorText': 'Polizei', 'url': 'https://www..."
64843,8105478890,deu,False,2024-05-01,00:13:52,2024-05-01 00:13:52+00:00,2024-05-01T00:13:17Z,news,0.686275,https://www.zeit.de/politik/ausland/2024-05/ge...,...,Die Audioversion dieses Artikels wurde künstli...,"{'uri': 'zeit.de', 'dataType': 'news

In [3]:
full_year["body"] = full_year["body"].apply(lambda x: x.replace("\n", ""))

In [4]:
full_year

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,...,body,source,authors,image,eventUri,sentiment,wgt,relevance,keywords,links
0,2024-12-590516688,deu,False,2024-12-31,23:48:05,2024-12-31 23:48:05+00:00,2024-12-31T23:35:24Z,news,0.000000,https://www.zeit.de/news/2025-01/01/feuerwehr-...,...,Kurz vor dem Jahreswechsel ist in Hamburg ein ...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/feuerwehr-...,None,NaN,473384885,1,"[News, Silvesternacht, Billstedt, Hamburg, Feu...","[{'anchorText': 'Feuerwehr', 'url': 'https://w..."
1,8481124955,deu,False,2024-12-31,23:26:31,2024-12-31 23:26:31+00:00,2024-12-31T23:25:31Z,news,0.968627,https://www.zeit.de/news/2025-01/01/zweite-squ...,...,"Das Drama um scheinbar harmlose, jedoch am End...","{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/zweite-squ...,eng-10211662,NaN,473383591,1,"[News, Netflix-Welthit, Netflix, Squid, Kinder...","[{'anchorText': 'Netflix', 'url': 'https://www..."
2,8481124174,deu,False,2024-12-31,23:25:05,2024-12-31 23:25:05+00:00,2024-12-31T23:24:01Z,news,0.000000,https://www.zeit.de/gesellschaft/zeitgeschehen...,...,Vor der Silvesternacht in Berlin hat die Poliz...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/gesellschaft/zeitgeschehen...,None,NaN,473383505,1,"[Gesellschaft, Jahreswechsel, Silvester, Feuer...","[{'anchorText': 'Polizei', 'url': 'https://www..."
3,8481124175,deu,False,2024-12-31,23:24:34,2024-12-31 23:24:34+00:00,2024-12-31T23:24:01Z,news,0.000000,https://www.zeit.de/news/2025-01/01/haus-in-hu...,...,Ein Einfamilienhaus ist in der Silvesternacht ...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/haus-in-hu...,None,NaN,473383474,1,"[News, Großeinsatz der Feuerwehr, Hude, Landkr...","[{'anchorText': 'Feuerwehr', 'url': 'https://w..."
4,8481116703,deu,False,2024-12-31,23:14:35,2024-12-31 23:14:35+00:00,2024-12-31T23:12:42Z,news,0.945098,https://www.zeit.de/news/2025-01/01/polen-uebe...,...,Polen hat zum Jahreswechsel den alle sechs Mon...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/polen-uebe...,eng-10209752,NaN,473382875,1,"[News, Nach sechs Monaten Ungarn, Donald Tusk,...","[{'anchorText': 'Polen', 'url': 'https://www.z..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64840,8105574979,deu,False,2024-05-01,02:23:35,2024-05-01 02:23:35+00:00,2024-05-01T02:22:46Z,news,0.800000,https://www.zeit.de/news/2024-05/01/jahrestag-...,...,Die Zusammenfassung für diesen Artikel kann le...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2024-05/01/jahrestag-...,deu-1781817,NaN,452226215,1,"[News, EU, Annalena Baerbock, Radosław Sikorsk...","[{'anchorText': 'Annalena Baerbock', 'url': 'h..."
64841,8105574980,deu,False,2024-05-01,02:23:18,2024-05-01 02:23:18+00:00,2024-05-01T02:22:46Z,news,0.588235,https://www.zeit.de/politik/ausland/2024-05/us...,...,Die Audioversion dieses Artikels wurde künstli...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/politik/ausland/2024-05/mi...,deu-1781792,NaN,452226198,1,"[Politik, US-Repräsentantenhaus, US-Repräsenta...",None
64842,8105568694,deu,False,2024-05-01,02:15:24,2024-05-01 02:15:24+00:00,2024-05-01T02:14:45Z,news,0.968627,https://www.zeit.de/news/2024-05/01/sek-einsat...,...,Die Zusammenfassung für diesen Artikel kann le...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2024-05/01/sek-einsat...,deu-1781828,NaN,452225724,1,"[News, Extremismus, Velbert, Düsseldorf, Poliz...","[{'anchorText': 'Polizei', 'url': 'https://www..."
64843,8105478890,deu,False,2024-05-01,00:13:52,2024-05-01 00:13:52+00:00,2024-05-01T00:13:17Z,news,0.686275,https://www.zeit.de/politik/ausland/2024-05/ge...,...,Die Audioversion dieses Artikels wurde künstli...,"{'uri': 'zeit.de', 'dataType': 'news

In [5]:
full_year.to_csv("../input-data/zeit-temp-data.csv")